In [5]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]
data.head(20)

,ID,Year,Sales,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,const
8,1,2021,257,0.010319,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
9,1,2022,183,0.005938,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
10,1,2023,177,0.002822,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
21,2,2023,3,0.000048,Aiways,U6,405,3.757100e+05,214,34,SUV,C,CH,1.0
28,3,2019,222,0.040217,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
29,3,2020,491,0.034512,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
30,3,2021,260,0.010440,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
31,3,2022,538,0.017456,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
32,3,2023,68,0.001084,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
41,4,2021,84,0.003373,Audi,e-tron GT,472,1.081416e+06,522,17,Sedan,F,DE,1.0


In [7]:
# Copy the dataframe
df2 = data.copy()

In [8]:
# Creating dummies for each segment
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)

# Creating dummies for each year
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)

# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)

In [9]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

In [10]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'Chargetime']]
dummies = df2[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['HP']]

In [11]:
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='unadjusted')

In [12]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -0.3398
Estimator:                    IV-2SLS   Adj. R-squared:                -0.4261
No. Observations:                 348   F-statistic:                    130.38
Date:                Sat, Mar 09 2024   P-value (F-stat)                0.0000
Time:                        16:42:51   Distribution:                 chi2(21)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -3.6041     1.4074    -2.5609     0.0104     -6.3626     -0.8457
Range         -0.0010     0.0088    -0.1096     0.9127     -0.0183      0.0163
Chargetime    -0.0585     0.0274    -2.1372     0.0326     -0.1122     -0.0049
Segment_B     -1.6423     1.4961    -1.0977     0.2723     -4.5746      1.2900
Segment_C     -1.7300     2.1494    -0.8049     0.4209     -5.9428      2.4827
Segment_D     -3.9883     3.9266    -1.0157     0.3098     -11.684      3.7078
Segment_E     -8.2860     6.7587    -1.2260     0.2202     -21.533      4.9609
Segment_F     -14.339     12.696    -1.1294     0.2587     -39.223      10.545
Segment_M     -6.5741     4.9067    -1.3398     0.1803     -16.191      3.0429
Segment_J     -3.3974     4.1643    -0.8158     0.4146     -11.559      4.7645
Year_2014      0.5773     1.4600     0.3954     0.6925     -2.2842      3.4388
Year_2015      0.7735     1.5299     0.5056     0.6131     -2.2250      3.7721
Year_2016      0.6827     1.4480     0.4715     0.6373     -2.1553      3.5207
Year_2017      0.5967     1.4147     0.4218     0.6732     -2.1760      3.3694
Year_2018      0.1352     1.4178     0.0954     0.9240     -2.6436      2.9141
Year_2019     -0.4279     1.2710    -0.3366     0.7364     -2.9190      2.0633
Year_2020     -1.6908     1.1520    -1.4677     0.1422     -3.9487      0.5671
Year_2021     -2.7952     1.1790    -2.3708     0.0178     -5.1060     -0.4844
Year_2022     -3.1364     1.1678    -2.6857     0.0072     -5.4252     -0.8475
Year_2023     -3.7766     1.1585    -3.2598     0.0011     -6.0472     -1.5059
China          1.3497     2.2485     0.6003     0.5483     -3.0573      5.7566
Price       1.578e-05  1.669e-05     0.9455     0.3444  -1.693e-05   4.849e-05
==============================================================================

Endogenous: Price
Instruments: HP
Unadjusted Covariance (Homoskedastic)
Debiased: False
"""